# Minimal dummy example with zerobnl

We consider the following two nodes graph as a minimal dummy example of two nodes exchanging a variable and randomly add or substract a given value from it:

<img src="./model.png" alt="Model" style="width: 500px;"/>

The above graph can be created using zerobnl as follow:

In [1]:
import os
from zerobnl import CoSim

In [2]:
sim = CoSim()

sim.create_meta_model("MetaBase", [("a", "unit")], [("b", "unit")])
sim.create_environment("EnvBase", "wrapper_base.py", "Dockerfile_base")

sim.add_node("Base0", "MetaBase", "EnvBase", init_values={"c": 0.50}, files=["empty_file_for_example_purpose.txt"])
sim.add_node("Base1", "MetaBase", "EnvBase", init_values={"c": 0.25})

sim.add_link("Base0", "b", "Base1", "a")
sim.add_link("Base1", "b", "Base0", "a")

sim.create_sequence([["Base0"], ["Base1"]])
sim.set_time_unit("seconds")
sim.create_steps([15] * 4 * 60)

This simulation is based on `wrapper_base.py`, you can use to it as template to create your own. 

In [3]:
sim.run()

INFO :: Simulation finished in 0 min and 6 sec


The first time you run this cell, it can take a long time, all the containers have to be build from scratch, once it's done it will run much faster.

If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [4]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,X,Base1,y
1,OUT,Base1,b
2,OUT,Base0,b
3,X,Base0,y


The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [5]:
for key, value in sim.get_results_by_pattern("OUT*Base0*").items():
    print(key)
    print(value)

INFO :: Matching results: Base0 - b


OUT||Base0||b
2000-01-01 00:00:15    0.00
2000-01-01 00:00:30   -0.25
2000-01-01 00:00:45   -0.50
2000-01-01 00:01:00    0.00
2000-01-01 00:01:15    0.25
2000-01-01 00:01:30    0.50
2000-01-01 00:01:45    0.75
2000-01-01 00:02:00    0.50
2000-01-01 00:02:15    0.50
2000-01-01 00:02:30    1.25
2000-01-01 00:02:45    1.00
2000-01-01 00:03:00    0.50
2000-01-01 00:03:15    0.75
2000-01-01 00:03:30    1.25
2000-01-01 00:03:45    1.75
2000-01-01 00:04:00    2.50
2000-01-01 00:04:15    2.00
2000-01-01 00:04:30    2.25
2000-01-01 00:04:45    1.75
2000-01-01 00:05:00    2.00
2000-01-01 00:05:15    1.75
2000-01-01 00:05:30    1.25
2000-01-01 00:05:45    1.00
2000-01-01 00:06:00    0.75
2000-01-01 00:06:15    1.00
2000-01-01 00:06:30    1.25
2000-01-01 00:06:45    1.00
2000-01-01 00:07:00    0.75
2000-01-01 00:07:15    1.00
2000-01-01 00:07:30    0.25
                       ... 
2000-01-01 00:52:45   -5.50
2000-01-01 00:53:00   -4.75
2000-01-01 00:53:15   -5.00
2000-01-01 00:53:30   -4.50
2000-0